## `builtin`

除`zipline`标准因子、过滤器、分类器外，增加`builtin`模块，将基础数据与标准`pipeline`整合，修改或模拟`quantopian`IDE功能。主要包括以下部分：
+ 自定义因子
+ 自定义过滤器
+ 自定义分类器

In [1]:
from zipline.pipeline.fundamentals.reader import Fundamentals
from zipline.pipeline.builtin import TradingDays,QTradableStocks
from zipline.research import run_pipeline, select_output_by
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage,Returns

### 因子

#### `SuccessiveYZ`
连续一字板数量（涨停、跌停）

**注意**统计连续一字，而非期间一字总数

In [2]:
from zipline.pipeline.builtin import SuccessiveYZ


def make_pipeline():
    window_length = 100
    yzzt, yzdt = SuccessiveYZ()
    return Pipeline(columns={
        '期间涨幅': Returns(window_length=window_length),
        '涨停个数': yzzt,
        '跌停个数': yzdt
    })


result = run_pipeline(make_pipeline(), '2018-4-24', '2018-4-27')
select_output_by(result, stock_codes=['603876', '603733'])

期间涨幅  涨停个数  跌停个数
2018-04-24 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0
2018-04-25 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0
2018-04-26 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0
2018-04-27 00:00:00+00:00 仙鹤股份(603733)   NaN   1.0   1.0
                          鼎胜新材(603876)   NaN   1.0   1.0

In [3]:
result = run_pipeline(make_pipeline(), '2018-2-5', '2018-2-9')
select_output_by(result, stock_codes=['600150', '000693', '600074'])

期间涨幅  涨停个数  跌停个数
2018-02-05 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          ST保千里(600074) -0.723773   1.0   1.0
                          *ST船舶(600150) -0.004439   1.0   1.0
2018-02-06 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          ST保千里(600074) -0.737247   1.0   1.0
                          *ST船舶(600150) -0.010826   1.0   1.0
2018-02-07 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          ST保千里(600074) -0.750722   1.0   1.0
                          *ST船舶(600150) -0.010826   1.0   1.0
2018-02-08 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          ST保千里(600074) -0.763234   1.0   1.0
                          *ST船舶(600150) -0.014776   1.0   1.0
2018-02-09 00:00:00+00:00 *ST华泽(000693)  0.000000   1.0   1.0
                          ST保千里(600074) -0.751684   1.0   1.0
                          *ST船舶(600150)  0.003253   1.0   1.0

#### `NDays` 上市天数

In [4]:
from zipline.pipeline.builtin import NDays

In [5]:
def make_pipeline():
    ndays = NDays()
    return Pipeline(
        columns={
            '上市天数': ndays,
        }
    )

In [6]:
result = run_pipeline(make_pipeline(), '2018-1-20', '2018-1-26')

In [7]:
select_output_by(result,stock_codes=['600645','603103','603214','603876'])

上市天数
2018-01-22 00:00:00+00:00 中源协和(600645)  9029.0
                          横店影视(603103)   102.0
2018-01-23 00:00:00+00:00 中源协和(600645)  9030.0
                          横店影视(603103)   103.0
2018-01-24 00:00:00+00:00 中源协和(600645)  9031.0
                          横店影视(603103)   104.0
2018-01-25 00:00:00+00:00 中源协和(600645)  9032.0
                          横店影视(603103)   105.0
2018-01-26 00:00:00+00:00 中源协和(600645)  9033.0
                          横店影视(603103)   106.0

In [8]:
from zipline.pipeline.builtin import TradingDays

#### `TradingDays` 期间交易天数
+ 当天成交量大于0,有效交易
+ 需要指定`window_length`

In [9]:
def make_pipeline():
    t20 = TradingDays(window_length=20)
    t200 = TradingDays(window_length=200)
    return Pipeline(
        columns={
            '20天内有效交易天数': t20,
            '200天内有效交易天数': t200,           
        }, 
    )

In [10]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [11]:
select_output_by(result,'2018-04-23','2018-04-24',stock_codes=['000001','600645','600076'])

200天内有效交易天数  20天内有效交易天数
2018-04-23 00:00:00+00:00 平安银行(000001)        200.0        20.0
                          康欣新材(600076)        151.0         0.0
                          中源协和(600645)        127.0        20.0
2018-04-24 00:00:00+00:00 平安银行(000001)        200.0        20.0
                          康欣新材(600076)        150.0         0.0
                          中源协和(600645)        127.0        20.0

+ 600076期间停牌三个月，20日内无交易，但200天内存在交易
+ 600645在200天内有停牌，有效成交率不足90%
+ 000001每天正常交易

#### `SuccessiveSuspensionDays`
连续停牌天数

In [12]:
from zipline.pipeline.builtin import SuccessiveSuspensionDays

def make_pipeline():
    days_90 = SuccessiveSuspensionDays(window_length=90)
    return Pipeline(
        columns={
            '90天内连续停牌天数': days_90,
            '成交量': USEquityPricing.volume.latest,           
        }, 
    )

result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-27')
select_output_by(result,'2018-04-23','2018-04-24',stock_codes=['000001','600645','600076'])

90天内连续停牌天数          成交量
2018-04-23 00:00:00+00:00 平安银行(000001)         1.0   95860000.0
                          康欣新材(600076)         1.0          0.0
                          中源协和(600645)         1.0    6070000.0
2018-04-24 00:00:00+00:00 平安银行(000001)         1.0  107020000.0
                          康欣新材(600076)         1.0          0.0
                          中源协和(600645)         1.0    4820000.0

### 过滤器

#### `IsST`
+ 当前是否为ST状态

In [13]:
from zipline.pipeline.builtin import IsST

In [14]:
def make_pipeline():
    is_st = IsST()
    # 用于mask参数，非st计算结果为NaN
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], mask=is_st, window_length=200)
    return Pipeline(columns={
        '平均收盘': ma20,
    })

In [15]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [16]:
select_output_by(
    result, '2018-04-23', stock_codes=['600408', '600645', '600076'])

平均收盘
2018-04-23 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)      NaN
                          中源协和(600645)       NaN
2018-04-24 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)  3.41749
                          中源协和(600645)       NaN
2018-04-25 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)  3.40904
                          中源协和(600645)       NaN
2018-04-26 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)  3.39834
                          中源协和(600645)       NaN

In [17]:
def make_pipeline():
    is_st = IsST()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    # 用于screen参数，非st不会显示
    return Pipeline(
        columns={
            '平均收盘': ma20,
        }, 
        screen=is_st
    )

In [18]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [19]:
select_output_by(
    result, '2018-04-23', stock_codes=['600408', '600645', '600076'])

,,平均收盘
2018-04-24 00:00:00+00:00,*ST安泰(600408),2.3025
2018-04-25 00:00:00+00:00,*ST安泰(600408),2.2800
2018-04-26 00:00:00+00:00,*ST安泰(600408),2.2620


#### `IsNewShare` 次新股
+ days：上市天数小于指定天数，判定为次新股，默认90天

In [20]:
from zipline.pipeline.builtin import IsNewShare

In [21]:
def make_pipeline():
    ndays = NDays()
    return Pipeline(
        columns={
            '上市天数': ndays,
        }, 
        screen=IsNewShare()
    )

In [22]:
run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

上市天数
2018-04-20 00:00:00+00:00 华西证券(002926)  74.0
                          泰永长征(002927)  56.0
                          华夏航空(002928)  49.0
                          润建通信(002929)  50.0
                          宏川智慧(002930)  23.0
                          锋龙股份(002931)  17.0
                          天邑股份(300504)  21.0
                          彩讯股份(300634)  28.0
                          南京聚隆(300644)  73.0
                          科顺股份(300737)  85.0
                          明阳电路(300739)  78.0
                          御家汇(300740)   71.0
                          华宝股份(300741)  50.0
                          江苏租赁(600901)  50.0
                          湖南盐业(600929)  25.0
                          成都银行(601838)  79.0
                          倍加洁(603059)   49.0
                          养元饮品(603156)  67.0
                          爱婴室(603214)   21.0
                          振德医疗(603301)   8.0
                          华菱精工(603356)  86.0
                          南都物业(603506)  78.0
                          淳中科技(603516)  77.0
                          今创集团(603680)  52.0
                          中源家居(603709)  71.0
                          七一二(603712)   53.0
                          沃格光电(603773)   3.0
                          嘉友国际(603871)  73.0
                          鼎胜新材(603876)   2.0
                          天永智能(603895)  88.0
...                                      ...
2018-04-25 00:00:00+00:00 长城科技(603897)  15.0
2018-04-26 00:00:00+00:00 华西证券(002926)  80.0
                          泰永长征(002927)  62.0
                          华夏航空(002928)  55.0
                          润建通信(002929)  56.0
                          宏川智慧(002930)  29.0
                          锋龙股份(002931)  23.0
                          天邑股份(300504)  27.0
                          彩讯股份(300634)  34.0
                          南京聚隆(300644)  79.0
                          明阳电路(300739)  84.0
                          御家汇(300740)   77.0
                          华宝股份(300741)  56.0
                          江苏租赁(600901)  56.0
                          湖南盐业(600929)  31.0
                          成都银行(601838)  85.0
                          倍加洁(603059)   55.0
                          养元饮品(603156)  73.0
                          爱婴室(603214)   27.0
                          振德医疗(603301)  14.0
                          南都物业(603506)  84.0
                          淳中科技(603516)  83.0
                          今创集团(603680)  58.0
                          中源家居(603709)  77.0
                          七一二(603712)   59.0
                          仙鹤股份(603733)   6.0
                          沃格光电(603773)   9.0
                          嘉友国际(603871)  79.0
                          鼎胜新材(603876)   8.0
                          长城科技(603897)  16.0

[152 rows x 1 columns]

#### `QTradableStocks` 量化可交易股票

In [23]:
from zipline.pipeline.builtin import QTradableStocks

In [24]:
def make_pipeline():
    stocks = QTradableStocks()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            'ma20': ma20,
        }, 
        screen=stocks
    )

In [25]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [26]:
ds = result.index.get_level_values(0).unique()

In [27]:
for d in ds:
    print('在日期为{}时，有{}只股票符合'.format(d.date(), result.loc[d].shape[0]))

在日期为2018-04-20时，有2620只股票符合
在日期为2018-04-23时，有2622只股票符合
在日期为2018-04-24时，有2620只股票符合
在日期为2018-04-25时，有2622只股票符合
在日期为2018-04-26时，有2628只股票符合


#### `TopAverageAmount` & `TAA` 平均成交额前N位
+ 平均成交额排名前N位的股票。默认前500位

In [28]:
from zipline.pipeline.builtin import TAA

def make_pipeline():
    stocks = QTradableStocks()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            'ma20': ma20,
        }, 
        screen=TAA()
    )

result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')
result.loc['2018-04-20'].shape == (500,1)

True

#### `IsYZZT` & `IsYZDT`
+ 默认包含ST一字板
+ 如排除，设定`include_st=False`

In [29]:
from zipline.pipeline.builtin import IsYZZT
from zipline.pipeline.factors import DailyReturns
def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅':dr,
            'ma20': ma20,
        }, 
        screen=IsYZZT()
    )
# 只输出一字涨停部分
result = run_pipeline(make_pipeline(), '2017-9-20', '2017-9-24')
# 注意其中包含600228、000403等ST股票
select_output_by(result, stock_codes=['600228', '000403', '002893','300699'])

ma20        涨幅
2017-09-20 00:00:00+00:00 华通热力(002893)   13.603333  0.100295
                          光威复材(300699)   30.569231  0.099935
2017-09-21 00:00:00+00:00 华通热力(002893)   14.305000  0.099866
                          光威复材(300699)   32.381429  0.100098
2017-09-22 00:00:00+00:00 ST生化(000403)   31.007450  0.050081
                          华通热力(002893)   15.054000  0.099939
                          光威复材(300699)   34.324667  0.099928
                          *ST昌九(600228)  10.235500  0.049951

In [30]:
def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅':dr,
            'ma20': ma20,
        }, 
        # 不包含ST
        screen=IsYZZT(include_st=False)
    )
# 只输出一字涨停部分
result = run_pipeline(make_pipeline(), '2017-9-20', '2017-9-24')
# 注意，此时没有包含600228、000403等ST股票
select_output_by(result, stock_codes=['600228', '000403', '002893','300699'])

ma20        涨幅
2017-09-20 00:00:00+00:00 华通热力(002893)  13.603333  0.100295
                          光威复材(300699)  30.569231  0.099935
2017-09-21 00:00:00+00:00 华通热力(002893)  14.305000  0.099866
                          光威复材(300699)  32.381429  0.100098
2017-09-22 00:00:00+00:00 华通热力(002893)  15.054000  0.099939
                          光威复材(300699)  34.324667  0.099928

In [31]:
from zipline.pipeline.builtin import IsYZDT


def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅': dr,
            'ma20': ma20,
        },
        screen=IsYZDT())


# 只输出一字跌停部分
run_pipeline(make_pipeline(), '2018-4-24', '2018-4-26')

ma20        涨幅
2018-04-24 00:00:00+00:00 *ST华泽(000693)   7.1105 -0.049296
                          *ST三维(000755)   6.1080 -0.049149
                          万丰奥威(002085)   12.3480 -0.099919
                          ST尤夫(002427)   17.6160 -0.050237
                          ST龙力(002604)    7.8790 -0.050000
2018-04-25 00:00:00+00:00 *ST华泽(000693)   6.7385 -0.049383
                          万丰奥威(002085)   12.2300 -0.100269
                          ST尤夫(002427)   16.6925 -0.049900
                          ST龙力(002604)    7.7035 -0.050817
                          *ST船舶(600150)  18.2750 -0.049708
                          *ST罗顿(600209)   9.4695 -0.049757
2018-04-26 00:00:00+00:00 *ST华泽(000693)   6.3850 -0.049351
                          华信国际(002018)    5.3630 -0.100186
                          ST尤夫(002427)   15.8150 -0.050420
                          圣阳股份(002580)    8.3280 -0.100358
                          ST龙力(002604)    7.5150 -0.049713
                          *ST船舶(600150)  18.1480 -0.049846
                          *ST哈空(600202)   7.7000 -0.049342
                          *ST罗顿(600209)   9.3875 -0.049808

In [32]:
def make_pipeline():
    dr = DailyReturns()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=20)
    return Pipeline(
        columns={
            '涨幅': dr,
            'ma20': ma20,
        },
        # 排除st
        screen=IsYZDT(include_st=False))


# 输出不包含St一字跌停
run_pipeline(make_pipeline(), '2018-4-24', '2018-4-26')

ma20        涨幅
2018-04-24 00:00:00+00:00 万丰奥威(002085)  12.348 -0.099919
2018-04-25 00:00:00+00:00 万丰奥威(002085)  12.230 -0.100269
2018-04-26 00:00:00+00:00 华信国际(002018)   5.363 -0.100186
                          圣阳股份(002580)   8.328 -0.100358

#### 有关停复牌
当日复牌

![停牌表](./images/20180426_list.png)

In [33]:
# 可用于限定范围
from zipline.pipeline.filters import StaticSids

In [34]:
from zipline.pipeline.builtin import IsResumed, SuccessiveSuspensionDays

def make_pipeline():
    #target = StaticSids([2163,600051])
    dr = DailyReturns()
    return Pipeline(
        columns={
            '涨幅': dr,
            '停牌天数':SuccessiveSuspensionDays(include=True)
        },
        screen=IsResumed())


# 只输出连续停牌后当日复牌的股票涨跌幅
run_pipeline(make_pipeline(), '2018-4-26', '2018-4-27')

停牌天数        涨幅
2018-04-26 00:00:00+00:00 珠海中富(000659)    1.0  0.005348
                          华信国际(002018)   23.0 -0.100186
                          圣阳股份(002580)   89.0 -0.100358
                          *ST哈空(600202)   1.0 -0.049342
                          *ST狮头(600539)  63.0 -0.037037
                          苏美达(600710)     6.0  0.100885
                          宁波中百(600857)    2.0  0.100284
                          星湖科技(600866)   59.0  0.101149
                          渤海活塞(600960)   89.0  0.002714
                          *ST蓝科(601798)   1.0 -0.050157
2018-04-27 00:00:00+00:00 中航三鑫(002163)   89.0 -0.099476
                          艾格拉斯(002619)   50.0 -0.070000
                          宁波联合(600051)   69.0 -0.099487
                          *ST椰岛(600238)   1.0 -0.050078
                          *ST信通(600289)  13.0  0.003021
                          鹏起科技(600614)   78.0 -0.100394
                          *ST友好(600778)   1.0 -0.045652
                          文灿股份(603348)   89.0       NaN

### 分类器

股票分类涉及到地区、行业、概念，为简化处理，不再动态跟踪数据，而是使用静态方式简化。每次回测时，都使用最新的分类数据。如股票在2018-2-1，所处地区为上海，后由于变更注册地，2018-4-1所处地区更改为北京。在2018-2-1回测时，该股票使用的分类数据是最新数据，即上海；而2018-4-1回测时，使用的分类数据为北京。但这会造成一个问题，即在不同的时间，以同样策略回测同一期间的数据时，结果会不一致。请注意此类差异。

分类器直接使用`Fundamentals`容器类

In [35]:
def make_pipeline():
    dqfl = Fundamentals.info.region.latest
    return Pipeline(
        columns={
            'dqfl': dqfl,
        }, 
        screen=dqfl.element_of([0,1,22])
    )

In [36]:
run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

dqfl
2018-04-20 00:00:00+00:00 云南白药(000538)      1
                          靖远煤电(000552)     22
                          昆百大Ａ(000560)      1
                          美好置业(000667)      1
                          荣丰控股(000668)      0
                          三毛派神(000779)     22
                          甘肃电投(000791)     22
                          云铝股份(000807)      1
                          云南铜业(000878)      1
                          云内动力(000903)      1
                          兰州黄河(000929)     22
                          南天信息(000948)      1
                          锡业股份(000960)      1
                          银亿股份(000981)     22
                          皇台酒业(000995)     22
                          科华生物(002022)      0
                          思源电气(002028)      0
                          丽江旅游(002033)      1
                          云南能投(002053)      1
                          威 尔 泰(002058)     0
                          云南旅游(002059)      1
                          罗平锌电(002114)      1
                          中国海诚(002116)      0
                          中核钛白(002145)     22
                          汉钟精机(002158)      0
                          悦心健康(002162)      0
                          延华智能(002178)      0
                          海得控制(002184)      0
                          华天科技(002185)     22
                          二三四五(002195)      0
...                                       ...
2018-04-26 00:00:00+00:00 申通地铁(600834)      0
                          上海机电(600835)      0
                          界龙实业(600836)      0
                          海通证券(600837)      0
                          上海九百(600838)      0
                          上柴股份(600841)      0
                          上工申贝(600843)      0
                          丹化科技(600844)      0
                          宝信软件(600845)      0
                          同济科技(600846)      0
                          上海临港(600848)      0
                          华东电脑(600850)      0
                          海欣股份(600851)      0
                          博闻科技(600883)      1
                          国投电力(600886)     22
                          张江高科(600895)      0
                          太平洋(601099)       1
                          环旭电子(601231)      0
                          交通银行(601328)      0
                          *ST智慧(601519)     0
                          中国太保(601601)      0
                          上海医药(601607)      0
                          广电电气(601616)      0
                          上海电气(601727)      0
                          光大证券(601788)      0
                          *ST蓝科(601798)    22
                          中远海发(601866)      0
                          招商轮船(601872)      0
                          龙宇燃油(603003)      0
                          华贸物流(603128)      0

[1225 rows x 1 columns]

In [37]:
# 通过查询获取分类编码的含义
Fundamentals.region_cname(0)

'上海市'

In [38]:
Fundamentals.region_cname(22)

'甘肃省'